In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pickle
import re
from tqdm import tqdm

import nltk
from nltk import word_tokenize
from nltk import punkt
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
stoplist = set(stopwords.words('english'))
id_column = 'id'
missing_token = 'UNK'

In [ ]:
df = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.describe ()

In [ ]:
df.score.value_counts()

In [ ]:
count_list = [(df.score == '0.50').sum(), (df.score == '0.25').sum(), (df.score == '0.00').sum(), (df.score == '0.75').sum(), (df.score == '1.00').sum()]
label_list = list(df['score'].unique())
plt.figure(figsize = (10, 7))
plt.pie(count_list, labels = label_list, autopct = '%.2f %%', startangle = 90, explode = (0.1, 0.1, 0.0), textprops = {'fontsize': 12})
plt.title('Distribution of Scores', fontsize = 20)
plt.show()

In [ ]:
df['anchor'].value_counts()

In [ ]:
df['target'].value_counts()

In [ ]:
df['context'].value_counts()

In [ ]:
df['score'].value_counts()

# encode the response

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['score'] = le.fit(df['score'])

In [ ]:
df['score'].value_counts()

In [ ]:
y = df['score']

In [ ]:
pickle.dump(y, open('y.pkl','wb'))

In [ ]:
df.info()

## factorize context since not too many levels ~100 for 30k observations

In [ ]:
df['context'].nunique()

In [ ]:
df['context'] = df['context'].astype('category').cat.codes

In [ ]:
df.info()

In [ ]:
df.head(2)

## analyze the text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def getTextFeatures(T, Col, max_features=1000): #, ngrams=(1,2), verbose=True):
    #if verbose:
    #    print('processing: ', Col)
    vectorizer = TfidfVectorizer() #ngram_range=ngrams)
    X = vectorizer.fit_transform(T[Col])
    return X, vectorizer.get_feature_names()

In [ ]:
X_anc, feat_anc = getTextFeatures(df, 'anchor')

In [ ]:
X_tar, feat_tar = getTextFeatures(df, 'target')

In [ ]:
pickle.dump(X_anc, open('X_anc.pkl','wb'))

In [ ]:
pickle.dump(X_tar, open('X_tar.pkl','wb'))

In [ ]:
pickle.dump(feat_anc, open('feat_anc.pkl','wb'))

In [ ]:
pickle.dump(feat_tar, open('feat_tar.pkl','wb'))

In [ ]:
pickle.dump(df, open('df.pkl','wb'))

In [ ]:
print(X_anc)

In [ ]:
print(X_tar)

In [ ]:
df.info()

In [ ]:
from scipy.sparse import hstack, csr_matrix
from sklearn.preprocessing import StandardScaler

In [ ]:
#X_anc= X_anc.reshape(-1, 1)
#X_tar = X_tar.reshape(-1, 1)
#X_anc= set(X_anc)
#X_tar= set(X_tar)

X_ = hstack(X_anc, X_tar)

In [ ]:
X_anc2D = [list(item) for item in X_anc]

In [ ]:
X_tar2D = [list(item) for item in X_tar]

In [ ]:
X_ = hstack(X_anc2D, X_tar2D)

In [ ]:
X_ancD = X_anc.toarray()

In [ ]:
X_tarD = X_tar.toarray()

In [ ]:
X_ = hstack(X_ancD, X_tarD)

In [ ]:
X_anc_den = csr_matrix.toarray(X_anc, out='2-D')

In [ ]:
X_tar_den = csr_matrix.toarray(X_tar)

In [ ]:
X_ = hstack(X_ancD, X_tarD)

In [ ]:
X_2 = hstack(X_ancD, X_tarD)

In [ ]:
print(X_ancD)

In [ ]:
print(X_tarD)

In [ ]:
context_ = df['context'].reshape(-1, 1)
context = StandardScaler().fit_transform(context_)

In [ ]:
X_ = np.hstack(X_anc, X_tar)

In [ ]:
X_ = hstack((X_anc, X_tar)).tocsr()